<a href="https://colab.research.google.com/github/jjangmo91/flying-squirrel/blob/main/dem_clip_5186.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import geopandas as gpd

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import geemap

# WDPA 필터링
wdpa = ee.FeatureCollection("WCMC/WDPA/current/polygons")
park_id = 773
boundary_feature = wdpa.filter(ee.Filter.eq('WDPAID', park_id))
park_info = boundary_feature.first().getInfo()
park_name = park_info['properties']['NAME']
print(f"WDPA에서 경계를 성공적으로 찾았습니다: {park_name} (WDPAID: {park_id})")
park_geom = boundary_feature.geometry()

# GEE에서 NASADEM 30m DEM 불러오기
nasadem = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
clipped_nasadem = nasadem.clip(park_geom)

# Google Drive로 내보내기 (TIF 파일로 저장)
output_crs = 'EPSG:5186'
output_scale = 30
final_file_path = f'/content/drive/MyDrive/KNPS/Siberian flying squirrel/data/wdpaid_{park_id}_nasadem_5186.tif'

geemap.ee_export_image(
    clipped_nasadem,
    filename=final_file_path, # <-- 바로 저장될 전체 경로
    scale=output_scale,
    crs=output_crs,
    region=park_geom.bounds(),
    file_per_band=False
)

print(f"\n작업 완료! {final_file_path} 에 파일이 저장되었습니다.")